In [20]:
from langchain.prompts import PromptTemplate
from langchain.prompts.base import BaseOutputParser
from langchain import OpenAI, LLMChain
from langchain.chains.llm_for_loop import LLMForLoopChain

In [21]:
# First we make a chain that generates the list

In [13]:
from typing import Optional
import re
class ListOutputParser(BaseOutputParser):
    
    def __init__(self, regex: Optional[str] = None):
        self.regex=regex
    
    def parse(self, text: str) -> list:
        splits = [t for t in text.split("\n") if t]
        if self.regex is not None:
            splits = [re.match(self.regex, s).group(1) for s in splits]
        return splits

In [30]:
template = """You are a list maker. Your job is make lists given a certain user input.

The format of your lists should be:

```
List:
- Item 1
- Item 2
...
```

Begin!:

User input: {input}
List:"""
output_parser = ListOutputParser(regex="- (.*)")
prompt = PromptTemplate(template=template, input_variables=["input"], output_parser=output_parser)

chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

In [44]:
chain.predict_and_parse(input="top 5 ev companies")

['Tesla', 'Nissan', 'BMW', 'BYD', 'Volkswagen']

In [32]:
# Next we generate the chain that we run over each item

In [38]:
template = """For the following company, explain the origin of their name:

Company: {company}
Explanation of their name:"""
prompt = PromptTemplate(template=template, input_variables=["company"])

explanation_chain = LLMChain(llm=OpenAI(), prompt=prompt, verbose=True)

In [39]:
explanation_chain.predict(company="Tesla")



> Entering new chain...
Prompt after formatting:
For the following company, explain the origin of their name:

Company: Tesla
Explanation of their name:

> Finished chain.


'\n\nTesla is a company that specializes in electric cars and renewable energy. The company is named after Nikola Tesla, a Serbian-American inventor and electrical engineer who was born in the 19th century.'

In [40]:
# Now we combine them

In [41]:
for_loop_chain = LLMForLoopChain(llm_chain=chain, apply_chain=explanation_chain)

In [45]:
for_loop_chain.run_list(input="top 5 ev companies")



> Entering new chain...
Prompt after formatting:
For the following company, explain the origin of their name:

Company: Tesla
Explanation of their name:

> Finished chain.


> Entering new chain...
Prompt after formatting:
For the following company, explain the origin of their name:

Company: Nissan
Explanation of their name:

> Finished chain.


> Entering new chain...
Prompt after formatting:
For the following company, explain the origin of their name:

Company: BMW
Explanation of their name:

> Finished chain.


> Entering new chain...
Prompt after formatting:
For the following company, explain the origin of their name:

Company: BYD
Explanation of their name:

> Finished chain.


> Entering new chain...
Prompt after formatting:
For the following company, explain the origin of their name:

Company: Volkswagen
Explanation of their name:

> Finished chain.


['\n\nTesla was named after the Serbian-American inventor Nikola Tesla, who was known for his work in electricity and magnetism.',
 '\n\nNissan is a Japanese company, and their name comes from the Japanese word for "sun."',
 "\n\nThe company's name is an abbreviation for Bayerische Motoren Werke, which is German for Bavarian Motor Works.",
 '\n\nThe company\'s name is derived from the Chinese characters "Baiyu Dong", which literally mean "to catch the rain in the east". The name is a reference to the company\'s origins in the city of Shenzhen, in southeastern China.',
 '\n\nVolkswagen is a German car company. The word "Volkswagen" means "people\'s car" in German.']